# Motivation
Tools such as pandas assume that you can fit all of your data in RAM on your local machine. But data often lives in data warehouses, so we end up trying to:
* Download significant amounts of data onto our local machines and slurp it all up into memory
* Experiment with it and develop a proof-of-concept in Python
* When we need productionize it, rewrite the code in something more scalable

Data warehouses are typically stored on something bigger and more powerful than my laptop, plus the data is already there. They also have SQL backends that are highly optimized/extremely fast. So ideally we'd want to:
* Take a small sample of the data and load it using a local database such as DuckDB
* Develop in Python, but have it leverage the performance of SQL engines under-the-hood
* In production, use the same code to process large amounts of data on the data warehouse where it already lives, avoiding a code rewrite
* Get to keep all of your code, from data engineering to machine learning, in Python!

Ibis enables this by providing a unified dataframe interface to 20+ backends. The syntax is similar to dplyr in R, or Polars in Python (and Polars is also one of the supported backends). 

In [2]:
import ibis
from ibis import _

ibis.options.interactive = False  # This is the default.

# Explore game-level data

In [3]:
games = ibis.read_parquet("data/games.parquet")
games

DatabaseTable: ibis_read_parquet_wywsvl3r3zbvpkb5y7f6s7wkdy
  game_id           string
  event             string
  site              string
  date              string
  round             string
  white             string
  black             string
  result            string
  utc_date          string
  utc_time          string
  white_elo         string
  black_elo         string
  white_rating_diff string
  black_rating_diff string
  eco               string
  opening           string
  time_control      string
  termination       string
  white_title       string
  black_title       string

By default, you get a printable representation of the table schema, showing the name and data type of each column.

In [4]:
games.count().to_pyarrow().as_py()

423270

In [5]:
type(games)

ibis.expr.types.relations.Table

In [6]:
type(games.game_id)

ibis.expr.types.strings.StringColumn

If we call the `head` method to peek at the data, you'll notice that we don't actually see any data (yet).

In [7]:
games.head()

r0 := DatabaseTable: ibis_read_parquet_wywsvl3r3zbvpkb5y7f6s7wkdy
  game_id           string
  event             string
  site              string
  date              string
  round             string
  white             string
  black             string
  result            string
  utc_date          string
  utc_time          string
  white_elo         string
  black_elo         string
  white_rating_diff string
  black_rating_diff string
  eco               string
  opening           string
  time_control      string
  termination       string
  white_title       string
  black_title       string

Limit[r0, n=5]

So what's going on? Ibis has a deferred execution model. It builds up expressions/queries based on what we ask it to do, and then executes those expressions once actually necessary. It's lazy (or efficient).

To just see a few rows of the `games` table, we could ask for the results of `games.head()` as a `pandas.DataFrame`, `pyarrow.Table`, or `polars.DataFrame`:

In [8]:
games.head().to_pandas()

,game_id,event,site,date,round,white,black,result,utc_date,utc_time,white_elo,black_elo,white_rating_diff,black_rating_diff,eco,opening,time_control,termination,white_title,black_title
0,S3bP7JXA,Rated Blitz game,https://lichess.org/S3bP7JXA,2024.09.01,-,SihelyValahara,modze10,1-0,2024.09.01,09:21:36,2063,2072,+7,-7,C64,"Ruy Lopez: Classical Defense, Benelux Variation",180+0,Normal,None,None
1,X1OtRVZo,Rated Blitz game,https://lichess.org/X1OtRVZo,2024.09.01,-,certified,braquejnr,1-0,2024.09.01,09:21:36,1338,1325,+6,-6,C25,Vienna Game,180+0,Normal,None,None
2,tqye7Q9W,Rated Blitz game,https://lichess.org/tqye7Q9W,2024.09.01,-,Heavy_Metal,Kayra_16,1-0,2024.09.01,09:21:36,1290,1248,+5,-9,D00,Queen's Pawn Game: Chigorin Variation,180+0,Normal,None,None
3,841dKoEt,Rated Blitz game,https://lichess.org/841dKoEt,2024.09.01,-,sensorlife77,Superscarso,0-1,2024.09.01,09:21:36,2220,2236,-5,+6,A08,King's Indian Attack: French Variation,180+0,Normal,None,None
4,4qhCbnwD,Rated Bullet game,https://lichess.org/4qhCbnwD,2024.09.01,-,mesin1021,Sergey1184,1-0,2024.09.01,09:21:38,1526,1516,+6,-6,D06,Queen's Gambit Declined: Baltic Defense,120+1,Normal,None,None


Ibis also has an interactive mode. In this mode, Ibis will eagerly execute as much of the query as it needs to in order to show us 10 rows of the result.

("Eager" = not deferred/lazy)

In [9]:
ibis.options.interactive = True
games

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ game_id  ┃ event             ┃ site                         ┃ date       ┃ round  ┃ white           ┃ black              ┃ result ┃ utc_date   ┃ utc_time ┃ white_elo ┃ black_elo ┃ white_rating_diff ┃ black_rating_diff ┃ eco    ┃ opening                                         ┃ time_control ┃ termination  ┃ white_title ┃ black_title ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string   │ string            │ string                       │ string     │ string │ string          │ string             │ string │ string     │ string   │ string    │ string    │ string            │ string            │ string │ string                                          │ string       │ string       │ string      │ string      │
├──────────┼───────────────────┼──────────────────────────────┼────────────┼────────┼─────────────────┼────────────────────┼────────┼────────────┼──────────┼───────────┼───────────┼───────────────────┼───────────────────┼────────┼─────────────────────────────────────────────────┼──────────────┼──────────────┼─────────────┼─────────────┤
│ S3bP7JXA │ Rated Blitz game  │ ]8;id=562811;https://lichess.org/S3bP7JXA\https://lichess.org/S3bP7JXA]8;;\ │ 2024.09.01 │ -      │ SihelyValahara  │ modze10            │ 1-0    │ 2024.09.01 │ 09:21:36 │ 2063      │ 2072      │ +7                │ -7                │ C64    │ Ruy Lopez: Classical Defense, Benelux Variation │ 180+0        │ Normal       │ NULL        │ NULL        │
│ X1OtRVZo │ Rated Blitz game  │ ]8;id=167241;https://lichess.org/X1OtRVZo\https://lichess.org/X1OtRVZo]8;;\ │ 2024.09.01 │ -      │ certified       │ braquejnr          │ 1-0    │ 2024.09.01 │ 09:21:36 │ 1338      │ 1325      │ +6                │ -6                │ C25    │ Vienna Game                                     │ 180+0        │ Normal       │ NULL        │ NULL        │
│ tqye7Q9W │ Rated Blitz game  │ ]8;id=843016;https://lichess.org/tqye7Q9W\https://lichess.org/tqye7Q9W]8;;\ │ 2024.09.01 │ -      │ Heavy_Metal     │ Kayra_16           │ 1-0    │ 2024.09.01 │ 09:21:36 │ 1290      │ 1248      │ +5                │ -9                │ D00    │ Queen's Pawn Game: Chigorin Variation           │ 180+0        │ Normal       │ NULL        │ NULL        │
│ 841dKoEt │ Rated Blitz game  │ ]8;id=268356;https://lichess.org/841dKoEt\https://lichess.org/841dKoEt]8;;\ │ 2024.09.01 │ -      │ sensorlife77    │ Superscarso        │ 0-1    │ 2024.09.01 │ 09:21:36 │ 2220      │ 2236      │ -5                │ +6                │ A08    │ King's Indian Attack: French Variation          │ 180+0        │ Normal       │ NULL        │ NULL        │
│ 4qhCbnwD │ Rated Bullet game │ ]8;id=23072;https://lichess.org/4qhCbnwD\https://lichess.org/4qhCbnwD]8;;\ │ 2024.09.01 │ -      │ mesin1021       │ Sergey1184         │ 1-0    │ 2024.09.01 │ 09:21:38 │ 1526      │ 1516      │ +6                │ -6                │ D06    │ Queen's Gambit Declined: Baltic Defense         │ 120+1        │ Normal       │ NULL        │ NULL        │
│ KqBjz4jF │ Rated Rapid game  │ ]8;id=485190;https://lichess.org/KqBjz4jF\https://lichess.org/KqBjz4jF]8;;\ │ 2024.09.01 │ -      │ htfy            │ koorushfakharzadeh │ 0-1    │ 2024.09.01 │ 09:21:39 │ 1327      │ 1723      │ -2                │ +1                │ C50    │ Italian Game: Giuoco Pianissimo, Normal         │ 600+1        │ Normal   

So now we see a bit of the table itself, instead of a table schema or a query plan.

Note that, unlike pandas, databases/Parquet don't necessarily maintain a particular ordering of the rows (they don't have an index). 

# Ibis "verbs"—actions you can do to a table
We'll cover `select`, `drop`, `mutate`, `filter`, `order_by`, `aggregate`, and `group_by`. Time to dive in!
## Select and drop
`select` selects a subset of the columns in the original table. We can use strings of the column names:

In [10]:
games_selected_columns = games.select(
    "event",
    "result",
    "white_elo",
    "black_elo",
    "time_control",
    "termination",
    "white_title",
    "black_title",
)
games_selected_columns

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ event             ┃ result ┃ white_elo ┃ black_elo ┃ time_control ┃ termination  ┃ white_title ┃ black_title ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string            │ string │ string    │ string    │ string       │ string       │ string      │ string      │
├───────────────────┼────────┼───────────┼───────────┼──────────────┼──────────────┼─────────────┼─────────────┤
│ Rated Blitz game  │ 1-0    │ 2063      │ 2072      │ 180+0        │ Normal       │ NULL        │ NULL        │
│ Rated Blitz game  │ 1-0    │ 1338      │ 1325      │ 180+0        │ Normal       │ NULL        │ NULL        │
│ Rated Blitz game  │ 1-0    │ 1290      │ 1248      │ 180+0        │ Normal       │ NULL        │ NULL        │
│ Rated Blitz game  │ 0-1    │ 2220      │ 2236      │ 180+0        │ Normal       │ NULL        │ NULL        │
│ Rated Bullet game │ 1-0    │ 1526      │ 1516      │ 120+1        │ Normal       │ NULL        │ NULL        │
│ Rated Rapid game  │ 0-1    │ 1327      │ 1723      │ 600+1        │ Normal       │ NULL        │ NULL        │
│ Rated Blitz game  │ 1-0    │ 1552      │ 1650      │ 180+2        │ Normal       │ NULL        │ NULL        │
│ Rated Blitz game  │ 0-1    │ 2158      │ 2042      │ 180+2        │ Time forfeit │ NULL        │ WFM         │
│ Rated Bullet game │ 0-1    │ 1176      │ 1179      │ 60+0         │ Normal       │ NULL        │ NULL        │
│ Rated Blitz game  │ 0-1    │ 2154      │ 2166      │ 180+0        │ Time forfeit │ NULL        │ NULL        │
│ …                 │ …      │ …         │ …         │ …            │ …            │ …           │ …           │
└───────────────────┴────────┴───────────┴───────────┴──────────────┴──────────────┴─────────────┴─────────────┘

Or explicit references to the `Column` objects:

In [11]:
games_selected_columns.select(
    games_selected_columns.result,
    games_selected_columns.white_elo,
    games_selected_columns.black_elo,
)

┏━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ result ┃ white_elo ┃ black_elo ┃
┡━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string │ string    │ string    │
├────────┼───────────┼───────────┤
│ 1-0    │ 2063      │ 2072      │
│ 1-0    │ 1338      │ 1325      │
│ 1-0    │ 1290      │ 1248      │
│ 0-1    │ 2220      │ 2236      │
│ 1-0    │ 1526      │ 1516      │
│ 0-1    │ 1327      │ 1723      │
│ 1-0    │ 1552      │ 1650      │
│ 0-1    │ 2158      │ 2042      │
│ 0-1    │ 1176      │ 1179      │
│ 0-1    │ 2154      │ 2166      │
│ …      │ …         │ …         │
└────────┴───────────┴───────────┘

In this case, tab completion works to save typing "games_selected_columns" repeatedly. But Ibis also has a deferred operator `_`, which stands for the last table to the left of the current function call.

In [12]:
games_selected_columns.select(_.result, _.white_elo, _.black_elo)

┏━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ result ┃ white_elo ┃ black_elo ┃
┡━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string │ string    │ string    │
├────────┼───────────┼───────────┤
│ 1-0    │ 2063      │ 2072      │
│ 1-0    │ 1338      │ 1325      │
│ 1-0    │ 1290      │ 1248      │
│ 0-1    │ 2220      │ 2236      │
│ 1-0    │ 1526      │ 1516      │
│ 0-1    │ 1327      │ 1723      │
│ 1-0    │ 1552      │ 1650      │
│ 0-1    │ 2158      │ 2042      │
│ 0-1    │ 1176      │ 1179      │
│ 0-1    │ 2154      │ 2166      │
│ …      │ …         │ …         │
└────────┴───────────┴───────────┘

In [13]:
ibis.to_sql(games_selected_columns)

```sql
SELECT
  "t0"."event",
  "t0"."result",
  "t0"."white_elo",
  "t0"."black_elo",
  "t0"."time_control",
  "t0"."termination",
  "t0"."white_title",
  "t0"."black_title"
FROM "ibis_read_parquet_wywsvl3r3zbvpkb5y7f6s7wkdy" AS "t0"
```

Currently, we're using a DuckDB backend, but there's also a `dialect` keyword parameter to get the same query in other SQL dialects.

In [14]:
import ibis.selectors as s

games.select(s.contains("white"))

┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ white           ┃ white_elo ┃ white_rating_diff ┃ white_title ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string          │ string    │ string            │ string      │
├─────────────────┼───────────┼───────────────────┼─────────────┤
│ SihelyValahara  │ 2063      │ +7                │ NULL        │
│ certified       │ 1338      │ +6                │ NULL        │
│ Heavy_Metal     │ 1290      │ +5                │ NULL        │
│ sensorlife77    │ 2220      │ -5                │ NULL        │
│ mesin1021       │ 1526      │ +6                │ NULL        │
│ htfy            │ 1327      │ -2                │ NULL        │
│ samotos         │ 1552      │ +7                │ NULL        │
│ ZiguratZenMelle │ 2158      │ -8                │ NULL        │
│ ASowilem        │ 1176      │ -10               │ NULL        │
│ Amerekov-Chess  │ 2154      │ -5                │ NULL        │
│ …               │ …         │ …                 │ …           │
└─────────────────┴───────────┴───────────────────┴─────────────┘

`drop` is similar to `select`, but we specify which columns to _not_ include. 

### Exercise 1
How would you show all of the columns except the day (since this dataset only spans a couple of days) and the site (which seems redundant with `game_id`)?

In [ ]:
# Write your solution here


#### Solution

In [16]:
# %load solutions/nb01_ex01.py
games.drop("utc_date", "site")


┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ game_id  ┃ event             ┃ date       ┃ round  ┃ white           ┃ black              ┃ result ┃ utc_time ┃ white_elo ┃ black_elo ┃ white_rating_diff ┃ black_rating_diff ┃ eco    ┃ opening                                         ┃ time_control ┃ termination  ┃ white_title ┃ black_title ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string   │ string            │ string     │ string │ string          │ string             │ string │ string   │ string    │ string    │ string            │ string            │ string │ string                                          │ string       │ string       │ string      │ string      │
├──────────┼───────────────────┼────────────┼────────┼─────────────────┼────────────────────┼────────┼──────────┼───────────┼───────────┼───────────────────┼───────────────────┼────────┼─────────────────────────────────────────────────┼──────────────┼──────────────┼─────────────┼─────────────┤
│ S3bP7JXA │ Rated Blitz game  │ 2024.09.01 │ -      │ SihelyValahara  │ modze10            │ 1-0    │ 09:21:36 │ 2063      │ 2072      │ +7                │ -7                │ C64    │ Ruy Lopez: Classical Defense, Benelux Variation │ 180+0        │ Normal       │ NULL        │ NULL        │
│ X1OtRVZo │ Rated Blitz game  │ 2024.09.01 │ -      │ certified       │ braquejnr          │ 1-0    │ 09:21:36 │ 1338      │ 1325      │ +6                │ -6                │ C25    │ Vienna Game                                     │ 180+0        │ Normal       │ NULL        │ NULL        │
│ tqye7Q9W │ Rated Blitz game  │ 2024.09.01 │ -      │ Heavy_Metal     │ Kayra_16           │ 1-0    │ 09:21:36 │ 1290      │ 1248      │ +5                │ -9                │ D00    │ Queen's Pawn Game: Chigorin Variation           │ 180+0        │ Normal       │ NULL        │ NULL        │
│ 841dKoEt │ Rated Blitz game  │ 2024.09.01 │ -      │ sensorlife77    │ Superscarso        │ 0-1    │ 09:21:36 │ 2220      │ 2236      │ -5                │ +6                │ A08    │ King's Indian Attack: French Variation          │ 180+0        │ Normal       │ NULL        │ NULL        │
│ 4qhCbnwD │ Rated Bullet game │ 2024.09.01 │ -      │ mesin1021       │ Sergey1184         │ 1-0    │ 09:21:38 │ 1526      │ 1516      │ +6                │ -6                │ D06    │ Queen's Gambit Declined: Baltic Defense         │ 120+1        │ Normal       │ NULL        │ NULL        │
│ KqBjz4jF │ Rated Rapid game  │ 2024.09.01 │ -      │ htfy            │ koorushfakharzadeh │ 0-1    │ 09:21:39 │ 1327      │ 1723      │ -2                │ +1                │ C50    │ Italian Game: Giuoco Pianissimo, Normal         │ 600+1        │ Normal       │ NULL        │ NULL        │
│ 1uB4YHHH │ Rated Blitz game  │ 2024.09.01 │ -      │ samotos         │ tamir71            │ 1-0    │ 09:21:39 │ 1552      │ 1650      │ +7                │ -8                │ D30    │ Queen's Gambit Declined                         │ 180+2        │ Normal       │ NULL        │ NULL        │
│ 81WWusNZ │ Rated Blitz game  │ 2024.09.01 │ -      │ ZiguratZenMelle │ BT21PJM            │ 0-1    │ 09:21:39 │ 2158      │ 2042      │ -8                │ +8                │ D17    │ Slav Defense: Czech Variation, Krause Attack    │ 180+2        │ Time forfeit │ NULL        │ WFM         │
│ RGz9ttRa │ Rated Bullet game │ 2024.09.01 │ -      │ ASowilem        │ Egorwin            │ 0-1    │ 09:21:41 │ 1176      │ 1179      │ -10               │ +5     

## Mutate
While `select` and `drop` subtract columns, `mutate` can add them! We can create a new column as a function of existing column(s).

Elo ratings in chess are a metric of the strength of a player, with higher ratings signifying stronger players. For the sake of an example, let's assume that rating is more widely understood than Elo, so we want new columns labelled white/black_rating. And we definitely want it to be of data type int, not string. (`try_cast` is like `cast`, but doesn't fail on NULLs or other bad apples.)

In [17]:
games_selected_columns = games_selected_columns.mutate(
    white_rating=_.white_elo.try_cast(int)
)
games_selected_columns

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ event             ┃ result ┃ white_elo ┃ black_elo ┃ time_control ┃ termination  ┃ white_title ┃ black_title ┃ white_rating ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string            │ string │ string    │ string    │ string       │ string       │ string      │ string      │ int64        │
├───────────────────┼────────┼───────────┼───────────┼──────────────┼──────────────┼─────────────┼─────────────┼──────────────┤
│ Rated Blitz game  │ 1-0    │ 2063      │ 2072      │ 180+0        │ Normal       │ NULL        │ NULL        │         2063 │
│ Rated Blitz game  │ 1-0    │ 1338      │ 1325      │ 180+0        │ Normal       │ NULL        │ NULL        │         1338 │
│ Rated Blitz game  │ 1-0    │ 1290      │ 1248      │ 180+0        │ Normal       │ NULL        │ NULL        │         1290 │
│ Rated Blitz game  │ 0-1    │ 2220      │ 2236      │ 180+0        │ Normal       │ NULL        │ NULL        │         2220 │
│ Rated Bullet game │ 1-0    │ 1526      │ 1516      │ 120+1        │ Normal       │ NULL        │ NULL        │         1526 │
│ Rated Rapid game  │ 0-1    │ 1327      │ 1723      │ 600+1        │ Normal       │ NULL        │ NULL        │         1327 │
│ Rated Blitz game  │ 1-0    │ 1552      │ 1650      │ 180+2        │ Normal       │ NULL        │ NULL        │         1552 │
│ Rated Blitz game  │ 0-1    │ 2158      │ 2042      │ 180+2        │ Time forfeit │ NULL        │ WFM         │         2158 │
│ Rated Bullet game │ 0-1    │ 1176      │ 1179      │ 60+0         │ Normal       │ NULL        │ NULL        │         1176 │
│ Rated Blitz game  │ 0-1    │ 2154      │ 2166      │ 180+0        │ Time forfeit │ NULL        │ NULL        │         2154 │
│ …                 │ …      │ …         │ …         │ …            │ …            │ …           │ …           │            … │
└───────────────────┴────────┴───────────┴───────────┴──────────────┴──────────────┴─────────────┴─────────────┴──────────────┘

And then we don't need the old Elo column.

In [18]:
games_selected_columns = games_selected_columns.drop(_.white_elo)

### Method Chaining
We can build up more complicated queries by chaining together Ibis methods. The output of many Ibis methods is a table, just like the original `games` table. So we can continue calling table methods until we're satisfied (or until we end up with something that _isn't_ a table).

In [19]:
games_selected_columns = games_selected_columns.mutate(
    black_rating=_.black_elo.try_cast(int)
).drop("black_elo")
games_selected_columns

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ event             ┃ result ┃ time_control ┃ termination  ┃ white_title ┃ black_title ┃ white_rating ┃ black_rating ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string            │ string │ string       │ string       │ string      │ string      │ int64        │ int64        │
├───────────────────┼────────┼──────────────┼──────────────┼─────────────┼─────────────┼──────────────┼──────────────┤
│ Rated Blitz game  │ 1-0    │ 180+0        │ Normal       │ NULL        │ NULL        │         2063 │         2072 │
│ Rated Blitz game  │ 1-0    │ 180+0        │ Normal       │ NULL        │ NULL        │         1338 │         1325 │
│ Rated Blitz game  │ 1-0    │ 180+0        │ Normal       │ NULL        │ NULL        │         1290 │         1248 │
│ Rated Blitz game  │ 0-1    │ 180+0        │ Normal       │ NULL        │ NULL        │         2220 │         2236 │
│ Rated Bullet game │ 1-0    │ 120+1        │ Normal       │ NULL        │ NULL        │         1526 │         1516 │
│ Rated Rapid game  │ 0-1    │ 600+1        │ Normal       │ NULL        │ NULL        │         1327 │         1723 │
│ Rated Blitz game  │ 1-0    │ 180+2        │ Normal       │ NULL        │ NULL        │         1552 │         1650 │
│ Rated Blitz game  │ 0-1    │ 180+2        │ Time forfeit │ NULL        │ WFM         │         2158 │         2042 │
│ Rated Bullet game │ 0-1    │ 60+0         │ Normal       │ NULL        │ NULL        │         1176 │         1179 │
│ Rated Blitz game  │ 0-1    │ 180+0        │ Time forfeit │ NULL        │ NULL        │         2154 │         2166 │
│ …                 │ …      │ …            │ …            │ …           │ …           │            … │            … │
└───────────────────┴────────┴──────────────┴──────────────┴─────────────┴─────────────┴──────────────┴──────────────┘

## Filter
Back to verbs. `select` selected columns, `drop` dropped columns, and now `filter` filters rows based on some condition.

Suppose we're interested only in games where white is a pretty strong player.

In [20]:
games_selected_columns.filter(_.white_rating >= 2200)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ event                                                          ┃ result  ┃ time_control ┃ termination  ┃ white_title ┃ black_title ┃ white_rating ┃ black_rating ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string                                                         │ string  │ string       │ string       │ string      │ string      │ int64        │ int64        │
├────────────────────────────────────────────────────────────────┼─────────┼──────────────┼──────────────┼─────────────┼─────────────┼──────────────┼──────────────┤
│ Rated Blitz game                                               │ 0-1     │ 180+0        │ Normal       │ NULL        │ NULL        │         2220 │         2236 │
│ Rated Blitz game                                               │ 0-1     │ 180+0        │ Normal       │ NULL        │ NULL        │         2499 │         2659 │
│ Rated Blitz game                                               │ 1-0     │ 180+0        │ Normal       │ NULL        │ NULL        │         2526 │         2351 │
│ Rated Blitz game                                               │ 0-1     │ 180+0        │ Normal       │ NULL        │ WFM         │         2302 │         2259 │
│ Rated Blitz game                                               │ 1/2-1/2 │ 180+2        │ Normal       │ NULL        │ NULL        │         2318 │         2317 │
│ Rated Bullet game                                              │ 1-0     │ 60+0         │ Normal       │ NULL        │ NULL        │         2211 │         2121 │
│ Rated Blitz game                                               │ 0-1     │ 180+0        │ Normal       │ CM          │ NULL        │         2320 │         2209 │
│ Rated Blitz game                                               │ 1-0     │ 180+0        │ Time forfeit │ FM          │ NULL        │         2424 │         2465 │
│ Rated Blitz game                                               │ 1-0     │ 180+2        │ Time forfeit │ NULL        │ FM          │         2435 │         2486 │
│ Rated Rapid tournament https://lichess.org/tournament/wwLvVRtn │ 1-0     │ 600+0        │ Normal       │ NULL        │ NULL        │         2339 │         2270 │
│ …                                                              │ …       │ …            │ …            │ …           │ …           │            … │            … │
└────────────────────────────────────────────────────────────────┴─────────┴──────────────┴──────────────┴─────────────┴─────────────┴──────────────┴──────────────┘

Turns out this condition is true in nearly 10% of these games:

In [22]:
games_selected_columns.filter(
    _.white_rating >= 2200
).count().to_pandas() / games_selected_columns.count().to_pandas()

0.09452831525976327

Conditions can be combined using an element-wise logical operator.

In [23]:
strong_games = games_selected_columns.filter(
    (_.white_rating >= 2200) & (_.black_rating >= 2200)
)

In [24]:
strong_games.count().to_pandas() / games_selected_columns.count().to_pandas()

0.07500649703498949

Or by listing the component conditions as separate arguments (this is still a logical `and`).

In [25]:
games_selected_columns.filter(_.white_rating >= 2200, _.result == "0-1")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ event                                                          ┃ result ┃ time_control ┃ termination ┃ white_title ┃ black_title ┃ white_rating ┃ black_rating ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string                                                         │ string │ string       │ string      │ string      │ string      │ int64        │ int64        │
├────────────────────────────────────────────────────────────────┼────────┼──────────────┼─────────────┼─────────────┼─────────────┼──────────────┼──────────────┤
│ Rated Blitz game                                               │ 0-1    │ 180+0        │ Normal      │ NULL        │ NULL        │         2220 │         2236 │
│ Rated Blitz game                                               │ 0-1    │ 180+0        │ Normal      │ NULL        │ NULL        │         2499 │         2659 │
│ Rated Blitz game                                               │ 0-1    │ 180+0        │ Normal      │ NULL        │ WFM         │         2302 │         2259 │
│ Rated Blitz game                                               │ 0-1    │ 180+0        │ Normal      │ CM          │ NULL        │         2320 │         2209 │
│ Rated Rapid game                                               │ 0-1    │ 600+0        │ Normal      │ FM          │ NULL        │         2530 │         2453 │
│ Rated Blitz game                                               │ 0-1    │ 180+2        │ Normal      │ NULL        │ NULL        │         2300 │         2422 │
│ Rated Rapid game                                               │ 0-1    │ 600+0        │ Normal      │ NULL        │ NULL        │         2238 │         2172 │
│ Rated Blitz tournament https://lichess.org/tournament/vRH6uEjl │ 0-1    │ 180+0        │ Normal      │ NULL        │ NULL        │         2346 │         2087 │
│ Rated Rapid game                                               │ 0-1    │ 600+0        │ Normal      │ NULL        │ NULL        │         2437 │         2406 │
│ Rated Blitz game                                               │ 0-1    │ 180+0        │ Normal      │ NM          │ NULL        │         2477 │         2615 │
│ …                                                              │ …      │ …            │ …           │ …           │ …           │            … │            … │
└────────────────────────────────────────────────────────────────┴────────┴──────────────┴─────────────┴─────────────┴─────────────┴──────────────┴──────────────┘

## Order_by
And `order_by`, you guessed it, orders by the specified column (i.e. sorts the rows by the data in that column). Ibis enforces a default ascending order across all backends, even though some backends may have a native default of descending.

In [26]:
games_selected_columns.order_by(_.white_rating)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ event                                                          ┃ result  ┃ time_control ┃ termination ┃ white_title ┃ black_title ┃ white_rating ┃ black_rating ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string                                                         │ string  │ string       │ string      │ string      │ string      │ int64        │ int64        │
├────────────────────────────────────────────────────────────────┼─────────┼──────────────┼─────────────┼─────────────┼─────────────┼──────────────┼──────────────┤
│ Rated Rapid tournament https://lichess.org/tournament/z8aKjqAF │ 1-0     │ 900+5        │ Normal      │ NULL        │ NULL        │          400 │          427 │
│ Rapid swiss https://lichess.org/swiss/qDKFLMiJ                 │ 0-1     │ 600+5        │ Normal      │ NULL        │ NULL        │          400 │          830 │
│ Rated Blitz tournament https://lichess.org/tournament/hIz5EyO9 │ 1/2-1/2 │ 240+0        │ Normal      │ NULL        │ NULL        │          400 │          700 │
│ Rated Blitz tournament https://lichess.org/tournament/OCQKNKbv │ 0-1     │ 180+2        │ Normal      │ NULL        │ NULL        │          400 │          787 │
│ Rated Blitz tournament https://lichess.org/tournament/OCQKNKbv │ 1-0     │ 180+2        │ Normal      │ NULL        │ NULL        │          400 │         1554 │
│ Rated Blitz game                                               │ 1/2-1/2 │ 420+0        │ Normal      │ NULL        │ NULL        │          400 │         1031 │
│ Rated Blitz tournament https://lichess.org/tournament/hIz5EyO9 │ 1/2-1/2 │ 240+0        │ Normal      │ NULL        │ NULL        │          400 │          722 │
│ Rated Blitz tournament https://lichess.org/tournament/ImRR8iCE │ 0-1     │ 300+0        │ Normal      │ NULL        │ NULL        │          400 │         1584 │
│ Rated Rapid game                                               │ 0-1     │ 600+0        │ Normal      │ NULL        │ NULL        │          400 │          509 │
│ Rated Blitz tournament https://lichess.org/tournament/ImRR8iCE │ 0-1     │ 300+0        │ Normal      │ NULL        │ NULL        │          400 │          856 │
│ …                                                              │ …       │ …            │ …           │ …           │ …           │            … │            … │
└────────────────────────────────────────────────────────────────┴─────────┴──────────────┴─────────────┴─────────────┴─────────────┴──────────────┴──────────────┘

In [27]:
games_selected_columns.order_by(_.white_rating.desc())

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ event             ┃ result  ┃ time_control ┃ termination  ┃ white_title ┃ black_title ┃ white_rating ┃ black_rating ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string            │ string  │ string       │ string       │ string      │ string      │ int64        │ int64        │
├───────────────────┼─────────┼──────────────┼──────────────┼─────────────┼─────────────┼──────────────┼──────────────┤
│ Rated Bullet game │ 1/2-1/2 │ 60+0         │ Normal       │ GM          │ NULL        │         3301 │         2981 │
│ Rated Bullet game │ 0-1     │ 30+0         │ Normal       │ GM          │ NULL        │         3298 │         3039 │
│ Rated Bullet game │ 1-0     │ 30+0         │ Time forfeit │ GM          │ NULL        │         3298 │         3038 │
│ Rated Bullet game │ 0-1     │ 60+0         │ Time forfeit │ GM          │ IM          │         3295 │         2967 │
│ Rated Bullet game │ 0-1     │ 60+0         │ Normal       │ GM          │ NULL        │         3294 │         2998 │
│ Rated Bullet game │ 1-0     │ 60+0         │ Normal       │ GM          │ IM          │         3292 │         2970 │
│ Rated Bullet game │ 0-1     │ 30+0         │ Time forfeit │ GM          │ NULL        │         3291 │         3040 │
│ Rated Bullet game │ 1-0     │ 60+0         │ Time forfeit │ GM          │ NULL        │         3290 │         3002 │
│ Rated Bullet game │ 1-0     │ 60+0         │ Normal       │ GM          │ NULL        │         3288 │         3048 │
│ Rated Bullet game │ 1-0     │ 60+0         │ Time forfeit │ GM          │ NULL        │         3284 │         3053 │
│ …                 │ …       │ …            │ …            │ …           │ …           │            … │            … │
└───────────────────┴─────────┴──────────────┴──────────────┴─────────────┴─────────────┴──────────────┴──────────────┘

### Exercise 2
Show the 10 games with the highest white ratings where white nevertheless lost. 

In [28]:
# Write your solution here


#### Solution 2

In [30]:
# %load solutions/nb01_ex02.py
games_selected_columns.filter(_.result == "0-1").order_by(_.white_rating.desc())


┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ event             ┃ result ┃ time_control ┃ termination  ┃ white_title ┃ black_title ┃ white_rating ┃ black_rating ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ string            │ string │ string       │ string       │ string      │ string      │ int64        │ int64        │
├───────────────────┼────────┼──────────────┼──────────────┼─────────────┼─────────────┼──────────────┼──────────────┤
│ Rated Bullet game │ 0-1    │ 30+0         │ Normal       │ GM          │ NULL        │         3298 │         3039 │
│ Rated Bullet game │ 0-1    │ 60+0         │ Time forfeit │ GM          │ IM          │         3295 │         2967 │
│ Rated Bullet game │ 0-1    │ 60+0         │ Normal       │ GM          │ NULL        │         3294 │         2998 │
│ Rated Bullet game │ 0-1    │ 30+0         │ Time forfeit │ GM          │ NULL        │         3291 │         3040 │
│ Rated Rapid game  │ 0-1    │ 480+3        │ Time forfeit │ BOT         │ BOT         │         3233 │         2889 │
│ Rated Bullet game │ 0-1    │ 60+0         │ Normal       │ NULL        │ IM          │         3189 │         2851 │
│ Rated Bullet game │ 0-1    │ 60+0         │ Normal       │ NULL        │ NULL        │         3169 │         2928 │
│ Rated Bullet game │ 0-1    │ 60+0         │ Normal       │ NULL        │ NULL        │         3166 │         2915 │
│ Rated Bullet game │ 0-1    │ 60+0         │ Time forfeit │ NULL        │ NULL        │         3150 │         2911 │
│ Rated Bullet game │ 0-1    │ 60+0         │ Normal       │ NULL        │ NULL        │         3150 │         3042 │
│ …                 │ …      │ …            │ …            │ …           │ …           │            … │            … │
└───────────────────┴────────┴──────────────┴──────────────┴─────────────┴─────────────┴──────────────┴──────────────┘

### Exercise 3
Can you combine what you've learned so far with `cases` ( https://ibis-project.org/reference/expression-generic.html#ibis.expr.types.generic.Value.cases ) to show the 10 biggest upsets? By "upsets", we mean that the winner's rating is much lower than the loser's rating. Just ignore the games that ended in a draw (1/2-1/2).

In [ ]:
# Write your solution here


#### Solution 3

In [32]:
# %load solutions/nb01_ex03.py
expr = games_selected_columns.mutate(
    winner_margin=games_selected_columns.result.cases(
        (
            "0-1",
            games_selected_columns.black_rating - games_selected_columns.white_rating,
        ),
        (
            "1-0",
            games_selected_columns.white_rating - games_selected_columns.black_rating,
        ),
    )
).order_by(_.winner_margin)

expr


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ event                                                           ┃ result ┃ time_control ┃ termination  ┃ white_title ┃ black_title ┃ white_rating ┃ black_rating ┃ winner_margin ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string                                                          │ string │ string       │ string       │ string      │ string      │ int64        │ int64        │ int64         │
├─────────────────────────────────────────────────────────────────┼────────┼──────────────┼──────────────┼─────────────┼─────────────┼──────────────┼──────────────┼───────────────┤
│ Rated Bullet game                                               │ 1-0    │ 60+0         │ Time forfeit │ NULL        │ NULL        │          435 │         2853 │         -2418 │
│ Rated Blitz tournament https://lichess.org/tournament/FITLxmYD  │ 1-0    │ 180+2        │ Time forfeit │ NULL        │ NULL        │          500 │         2295 │         -1795 │
│ Rated Bullet tournament https://lichess.org/tournament/dcX8IvCg │ 0-1    │ 60+0         │ Time forfeit │ NULL        │ NULL        │         2381 │          642 │         -1739 │
│ Rated Bullet game                                               │ 0-1    │ 60+0         │ Normal       │ NULL        │ NULL        │         2089 │          435 │         -1654 │
│ Rated Blitz game                                                │ 1-0    │ 180+2        │ Normal       │ NULL        │ NULL        │          890 │         2504 │         -1614 │
│ Rated Bullet game                                               │ 0-1    │ 60+0         │ Normal       │ NULL        │ NULL        │         2902 │         1348 │         -1554 │
│ Rated Blitz tournament https://lichess.org/tournament/FITLxmYD  │ 0-1    │ 180+2        │ Normal       │ NULL        │ NULL        │         2078 │          568 │         -1510 │
│ Rated Blitz game                                                │ 1-0    │ 300+0        │ Normal       │ NULL        │ NULL        │          791 │         2266 │         -1475 │
│ Rated Rapid game                                                │ 1-0    │ 600+5        │ Normal       │ NULL        │ NULL        │         1006 │         2449 │         -1443 │
│ Rated Bullet tournament https://lichess.org/tournament/kGpCCMyH │ 1-0    │ 60+0         │ Time forfeit │ NULL        │ NULL        │          791 │         2199 │         -1408 │
│ …                                                               │ …      │ …            │ …            │ …           │ …           │            … │            … │             … │
└─────────────────────────────────────────────────────────────────┴────────┴──────────────┴──────────────┴─────────────┴─────────────┴──────────────┴──────────────┴───────────────┘

The deferred operator (`_`) really shines here, since it enables reference to a table object with no name. `"winner_margin"` isn't a column in `games_selected_columns`, so we wouldn't have been able to do this in one line of code otherwise.

In [33]:
ibis.to_sql(expr)

```sql
SELECT
  "t4"."event",
  "t4"."result",
  "t4"."time_control",
  "t4"."termination",
  "t4"."white_title",
  "t4"."black_title",
  "t4"."white_rating",
  "t4"."black_rating",
  CASE "t4"."result"
    WHEN '0-1'
    THEN "t4"."black_rating" - "t4"."white_rating"
    WHEN '1-0'
    THEN "t4"."white_rating" - "t4"."black_rating"
    ELSE NULL
  END AS "winner_margin"
FROM (
  SELECT
    "t3".*
    EXCLUDE ("black_elo")
  FROM (
    SELECT
      "t2"."event",
      "t2"."result",
      "t2"."black_elo",
      "t2"."time_control",
      "t2"."termination",
      "t2"."white_title",
      "t2"."black_title",
      "t2"."white_rating",
      TRY_CAST("t2"."black_elo" AS BIGINT) AS "black_rating"
    FROM (
      SELECT
        "t1".*
        EXCLUDE ("white_elo")
      FROM (
        SELECT
          "t0"."event",
          "t0"."result",
          "t0"."white_elo",
          "t0"."black_elo",
          "t0"."time_control",
          "t0"."termination",
          "t0"."white_title",
          "t0"."black_title",
          TRY_CAST("t0"."white_elo" AS BIGINT) AS "white_rating"
        FROM "ibis_read_parquet_wywsvl3r3zbvpkb5y7f6s7wkdy" AS "t0"
      ) AS "t1"
    ) AS "t2"
  ) AS "t3"
) AS "t4"
ORDER BY
  CASE "t4"."result"
    WHEN '0-1'
    THEN "t4"."black_rating" - "t4"."white_rating"
    WHEN '1-0'
    THEN "t4"."white_rating" - "t4"."black_rating"
    ELSE NULL
  END ASC
```

If you came up with a different solution, it's possible that the generated SQL (`ibis.to_sql()`) is slightly different. In practice, small differences in the generated SQL don't make a difference. Any modern SQL execution engine will optimize execution of queries containing variations of the same set of operations, and there will be no measurable performance difference.

## Aggregate

Ibis has several aggregate functions available to help summarize data.  All the old favorites are there: `mean`, `max`, `min`, `count`, `sum`...

You can aggregate a column by calling the method on that column:

In [ ]:
games_selected_columns.white_rating.mean()

There's also an `aggregate` function (also affectionately known as `agg`). Aggregates really shine when paired with a `group_by`.

# Group by

`group_by` creates groupings of rows that have the same value for one or more columns. But it doesn't do much on its own; instead, you can pair it an aggregate function to get a result:

In [ ]:
games_selected_columns.group_by("white_title").count().to_pandas()

Using just `agg` without any aggregate function specified gives us the distinct values (or pairs or sets of values) of the grouped column(s):

In [ ]:
games_selected_columns.group_by("white_title", "termination").agg().to_pandas()

Or we can pass an aggregate function to `agg`. This is also compatible with more method chaining after the `agg` call!

In [ ]:
games_selected_columns.group_by("white_title").agg(_.white_rating.mean()).order_by(
    ibis.desc("Mean(white_rating)")
)

With `agg`, we also don't have to stick with the default aggregate data column names:

In [ ]:
games_selected_columns.group_by("white_title").agg(
    average_white_rating=_.white_rating.mean()
).order_by(ibis.desc("average_white_rating"))

### Exercise 4
What was the distribution of results in games (number of white wins, black wins, and draws) involving at least one grandmaster ("GM")?

In [ ]:
# Write your solution here


#### Solution

In [ ]:
%load solutions/nb01_ex04.py

Let's compare that to games where both players were untitled.

In [ ]:
%load solutions/nb01_ex05.py

Finally, let's use `group_by` and `agg` to get a better understanding of the `time_control` column.

In [ ]:
split_index = games_selected_columns.time_control.find("+")
base_time = games_selected_columns.time_control.substr(0, split_index).try_cast(int)
increment = games_selected_columns.time_control.substr(split_index + 1).try_cast(int)
games_effective_time = games_selected_columns.mutate(
    effective_time_control=(base_time + 40 * increment) / 60
)

In [ ]:
expr = games_effective_time.filter(
    ~(_.event.contains("tournament")), ~(_.event.contains("swiss"))
).group_by("event")
expr.agg(
    min_eff_time=_.effective_time_control.min(),
    max_eff_time=_.effective_time_control.max(),
).order_by(_.max_eff_time)

In [ ]:
white_counts = games.white.value_counts()
white_counts.order_by(_.white_count.desc())

In [ ]:
black_counts = games.black.value_counts()
black_counts.order_by(_.black_count.desc())

In [ ]:
joined_counts = white_counts.outer_join(
    black_counts, (white_counts.white == black_counts.black)
)
expr = joined_counts.mutate(
    total_games=_.white_count.fill_null(0) + _.black_count.fill_null(0)
).order_by(_.total_games.desc())

expr

In [ ]:
ibis.to_sql(expr)